In [3]:
import pandas as pd
from emoji import UNICODE_EMOJI
import emoji
import regex as re
import tweepy
from tqdm import tqdm
import tensorflow_hub as hub
import tensorflow as tf
import lightgbm as lgbm
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import pickle
# loading the tensorflow universal sentence encoder 4 model


In [6]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


In [7]:
# setup API auhorizaion

# assign the values accordingly
consumer_key = "jfpVhd1zOxM6uqR1GjcQzEsns"
consumer_secret = "c4PdCb2fvIbFRHlgQJlT6k0cPlRUtl1E0akP5dSvmKz1NFUHmr"
access_token = "1250984376671670272-3wFUr43oEV9NUiPtv8f7ygVT74Rw3s"
access_token_secret = "2OyDgcbZWYbVwgvFJttSIEroJzmOOTwq9hGMGmVeRs9mg"

# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  
# set access to user's access key and access secret 
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [21]:
def clean(text):
    text = re.sub(r"'s", "", text) # remove possesive 's
    text = re.sub(r"http\S+", " ", text) # remove urls
    text = re.sub(r"RT ", " ", text) # remove rt
    text = re.sub(r"'[s]", "", text) # remove '
    text = re.sub(r"[^a-zA-Z\d\s]", " ", text) # remove special characters
    text = re.sub(r"[0-9]", " ", text) # remove number
    text = re.sub(r'\t', ' ', text) # remove tabs
    text = re.sub(r'\n', ' ', text) # remove line jump
    text = re.sub(r"\s+", " ", text) # remove extra white space
    text = text.strip()
    text = text.lower()
    return text

In [24]:
# count emojis in tweet text
def split_count(text):
    total_emoji = []
    data = re.findall(r'\X',text)
    flag = False
    for word in data:
        if word in emoji.UNICODE_EMOJI['en']:  
            total_emoji += [word] # total_emoji is a list of all emojis

    # remove from the given text the emojis
    for current in total_emoji:
        text = text.replace(current, '') 

    # return Counter(text.split() + total_emoji)
    return len(total_emoji)

In [25]:
# counts urls in tweet text
def countUrls(text):
    return len(re.findall(r"http\S+", text))

In [26]:
# counts hashtags in tweet text
def countHashtags(text):
    return len(re.findall(r"#[a-zA-Z]+", text))

In [27]:
# get list or count of followers for a specific user
def getFollowers(user, list=False):
    if list:
        return user.followers
    else:
        return user.followers_count

In [28]:
# get list or count of statuses for a specific user
def getStatuses(user, list=False):
    if list:
        return user.statuses
    else:
        return user.statuses_count

In [29]:
# get list or count of friends (people followed) for a specific user
def getFriends(user, list=False):
    if list:
        return user.friends
    else:
        return user.friends_count

In [30]:
# get list or count of friends (people followed) for a specific user
def getVerified(user):
    return user.verified

In [43]:
def prepDataForModel(predictionData):
    predictionData = predictionData[['emojiCount', 'followers', 'following', 'hashtagCount', 'urlCount', 'tweetCount', 'verified']]
    return predictionData

In [44]:
# this is how you construct a Prediction Set. Provide a list of user ids and the function will build your dataset
# save directory is where you would like your dataset to be saved to. 
# ex: r'/Users/lincolnrychecky/desktop/trainingDataBots.csv' saves the file as a csv named trainingDataBots.csv to my desktop

def build_prediction_set(userIds, saveDirectory):
    # construct the training set by calling predefined functions
    # feature lists
    ids = []
    human = []
    rawText = []
    text = []
    emojis = []
    followers = []
    following = []
    hashtags = []
    urls = []
    statuses = []
    verified = []
    
    for id in tqdm(userIds):
        # Put in a try-except block in case a forbidden user id is in there
        try:
            print(id)
            user = api.get_user(user_id=id)

            recentFiveTweets = api.user_timeline(user_id=id, 
                                    # 200 is the maximum allowed count
                                    count=11,
                                    include_rts = True,
                                    # Necessary to keep full_text 
                                    # otherwise only the first 140 words are extracted
                                    tweet_mode = 'extended'
                                    )
            fullText = ""
            for info in recentFiveTweets:
                fullText = fullText + info.full_text + " "
            cleanedText = clean(fullText)
            
            ids.append(id)
            emojis.append(split_count(fullText))
            followers.append(getFollowers(user))
            following.append(getFriends(user))
            hashtags.append(countHashtags(fullText))
            urls.append(countUrls(fullText))
            statuses.append(getStatuses(user))
            verified.append(getVerified(user))
            text.append(cleanedText)
            rawText.append(fullText)
            human.append(0)
        except:
            pass
        
    # assemble data dictionary
    dataDict = {
        "id": ids,
        "rawText": rawText,
        "cleanText": text,
        "emojiCount": emojis,
        "followers": followers,
        "following": following,
        "hashtagCount": hashtags,
        "urlCount": urls,
        "tweetCount": statuses,
        "verified": verified,
        "human": human
    }
    
    # turn dict into dataframe for export
    trainDataBot = pd.DataFrame.from_dict(dataDict)
    trainDataBot.to_csv(saveDirectory, index = False)
    return trainDataBot

In [45]:
x = build_prediction_set([1250474930178670593], r'/Users/raeganrychecky/desktop/predictiondata.csv' )

  0%|          | 0/1 [00:00<?, ?it/s]

1250474930178670593


100%|██████████| 1/1 [00:00<00:00,  2.53it/s]


In [46]:
mungeData = prepDataForModel(x)


In [51]:
mungeData

,emojiCount,followers,following,hashtagCount,urlCount,tweetCount,verified
0,0,138,75,0,6,740,False


In [52]:
scoringModel = pickle.load(open(r'/Users/raeganrychecky/desktop/CSCI5502/Political-Bot-Analysis/twitterBotModel/standard_model.sav', 'rb'))


In [53]:
predictions = scoringModel.predict(mungeData)

In [54]:
predictions

array([0.03112414])